In [3]:
import pandas as pd

##### READING THE MOVIE.CSV FILE

In [4]:
df = pd.read_csv("movies.csv")
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


##### REMOVING MOVIES WITH NO GENRES LISTED

In [18]:
df.loc[df["genres"]!="(no genres listed)"]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62417,209155,Santosh Subramaniam (2008),Action|Comedy|Romance
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama


##### REMOVING MOVIES WHONE NAMES CANNOT BE TYPED ON THE KEYBOARD
MOSTLY REMOVING THE RUSSIAN,CHINESE AND JAPANESE MOVIES WHICH CANNOT BE TYPED ON THE KEYBOARD

In [19]:
import re

In [59]:
cleaned_df = df.iloc[0:0,:]
for i in range(df.shape[0]):
    title = df.iloc[i,1]
    x = re.findall("[^a-zA-Z0-9()\'\",:\.\-!&\/?éàè+³#%\*½ ]", title)
    if x:
        continue
    else:
        cleaned_df = cleaned_df.append(df.iloc[i,:])

In [60]:
cleaned_df.shape

(60680, 3)

In [65]:
cleaned_df = cleaned_df.loc[cleaned_df["genres"]!= "(no genres listed)"]

In [66]:
cleaned_df.shape

(55804, 3)

##### SAVING THE CLEANED DATAFRAME

In [67]:
cleaned_df.to_csv("cleaned_movie.csv",index=False)

##### READING THE RATINGS.CSV FILE

In [68]:
ratings = pd.read_csv("ratings.csv")

In [69]:
ratings.shape

(25000095, 4)

In [70]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [71]:
ratings["userId"].unique()

array([     1,      2,      3, ..., 162539, 162540, 162541], dtype=int64)

##### DROPPING THE TIMESTAMP COLUMN AS IT DOESN'T HELP IN MAKING A RECOMMENDATION

In [73]:
ratings.drop(columns = ["timestamp"],inplace= True)
ratings.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [77]:
mov_id = cleaned_df["movieId"].tolist()

##### REMOVING MOVIES RATINGS OF MOVIES WHICH ARE NOT IN THE CLEANED MOVIES DATAFRAME

In [79]:
ratings = ratings.loc[ratings["movieId"].isin(mov_id)]

In [81]:
ratings.to_csv("cleaned_ratings.csv",index=False)